In [1]:
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def change_column_names(df, old_names, new_names):
    pass
    return df;

##input
##   string: 2013-02-22 21:38:45
##output: 
##   list: [u'2013', u'02', u'22', u'21', u'38', u'45']
def datetime_to_trace_time(timestamp):
    #strip double quotes
    timestamp = timestamp.replace('"', '')
    r_list = re.split('[- :]', timestamp)
    return [int(x) for x in r_list]

In [2]:
data_run='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'
df_runs = df_from_csv(data_run)
print type(df_runs)
df_runs.show(2)
df_runs.count()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|     "speed"|"workoutid"|    "id"|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544|6.7140000000|  167479013|13856798|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234|9.3492000000|  167479013|13856799|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
only showing top 2 rows



300000

In [3]:
sub_df = df_runs.limit(20)

In [4]:
#traces
import pandas as pd
import numpy as np
import re
import sys
import nbimporter
#for ordered dict for traces implementation
from collections import OrderedDict

import traces
from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

#kmeans
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_run='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [5]:
#get list of workoutids
#woutid_list = df_runs.select('"workoutid"').distinct()\
#    .rdd.map(lambda r:r[0]).collect()
#woutid_list

In [6]:
#class to create summary stats of workouts
class SummaryStat():
    data_points = ''
    new_col_names = []
    #list of tuples. 
    #   key: column name
    #   value: metric (sum/avg/etc)
    summary_conf = ''
    partition_name = ''
    def __init__(self, sql_df, summary_conf = [], \
                partition_name = ''):
        if partition_name:
            self.partition_name = partition_name
        else:
            self.partition_name = '"workoutid"'
        self.data_points = sql_df
        if not isinstance(sql_df, DataFrame):
            print 'please enter spark dataframe object'
            return
        #if summary_conf not defined, default to sum for all columns
        if not summary_conf:
            col_names = self.data_points.schema.names
            for i in col_names:
                summary_conf.append((i, 'sum'))
        else:
            if not isinstance(summary_conf, list):
                print 'check summary_conf type'
                return
            for element in summary_conf:
                if not isinstance(element, dict):
                    print 'check summary_conf type 2'
                    return
            self.summary_conf = summary_conf
        
    def describe(self):
        print self.data_points.limit(2)

    def get_new_col_names(self):
        return self.new_col_names
        
    def generate_summary(self):
        #rewnew new_col_names
        self.new_col_names = []
        #windows function test
        r_df = ''
        r_list_cols = []
        for dic in self.summary_conf:
            col_name = dic.keys()[0] #should only have 1 key per dictionary
            aggregation_type = dic[col_name]
            w = Window.partitionBy(self.partition_name)
            if aggregation_type == 'avg':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        avg(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())
                self.new_col_names.append(new_col_name)

            if aggregation_type == 'sum':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        sum(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())   
                self.new_col_names.append(new_col_name)

                
            if aggregation_type == 'max':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        max(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())    
                self.new_col_names.append(new_col_name)
                
            if aggregation_type == 'min':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        min(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct()) 
                self.new_col_names.append(new_col_name)
                
        for df in r_list_cols:
            if not r_df:
                r_df = df
            else:
                r_df = r_df.join(df, (r_df[self.partition_name]==df[self.partition_name]), 'inner')\
                    .drop(df[self.partition_name])
        return r_df
    


In [7]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def normalize(self, value):
        pass

In [8]:

#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols):
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name)
        scalerModel = scaler.fit(df)
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        index+=1
    return df, r_dict

# MinMaxScaler doesn't have originalMin(only supported in 2.0). made one above cell with wrapper class
# #generate normalized dataframe
# #keep the original column names
# #returns:
# #   (df, {col_name:scalerModel, col_name:scalerModel,....})
# def normalize_df(df, list_cols):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     for col_name in list_cols:
#         scaler = MinMaxScaler(inputCol=col_name, outputCol=tmp_col_name)
#         scalerModel = scaler.fit(df)
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         r_dict[col_name] = [scaler, scalerModel]
#     return df, r_dict

#input: (MLLIB KMEANS)
#   cols_to_cluster: list of column names to cluster
# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
#     vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
#         outputCol='features')
#     df_to_cluster = vecAssembler.transform(df.na.fill(0))
#     df_to_cluster = df_to_cluster.select('features')
#     df_to_cluster = df_to_cluster.rdd\
#         .map(lambda row : Vectors.dense([item for item in row]))
#     clusters = KMeans.train(df_to_cluster, num_clusters,\
#                            maxIterations = 10,\
#                         initializationMode="random")
    
#     return clusters


#ML KMEANS
#returns KMeanModel, df_to_cluster
def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
    vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
        outputCol='features')
    df_to_cluster = vecAssembler.transform(df.na.fill(0))
    df_to_cluster = df_to_cluster.select('features')
    #df_to_cluster = df_to_cluster.rdd\
     #   .map(lambda row : Vectors.dense([item for item in row]))
    kmeans = KMeans(k=num_clusters, seed=1)
    model = kmeans.fit(df_to_cluster)
    return model, df_to_cluster
    
#returns clusters. show cluster centerse by clusters.clusterCenters.
def cluster_df(df, cols_to_normalize, num_clusters = 2, return_sse = False, cols_to_cluster = []):
    if not cols_to_cluster:
        cols_to_cluster = cols_to_normalize
    df = change_type_cols_double(df, cols_to_normalize)
    df = vectorize_columns(df, cols_to_normalize)
    df, scalerModels = normalize_df(df, cols_to_cluster)
    r_obj = cluster_summary_df(df, cols_to_cluster, num_clusters)
    
    if return_sse:
        #calculate SSE for this kmean model
        pass
    
    
    #r_obj = [KMeanModel, new_df]
    return r_obj[0], r_obj[1], scalerModels

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers



# cluster section

In [9]:
#define columns to cluster on (they will be normalized)
#cols_to_cluster = ['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
#create summary stats for workouts

s_conf = [{'"heart_rate"': 'avg'}, {'"speed"': 'avg'}, {'"altitude"': 'avg'}]

#obj = SummaryStat(df_runs, summary_conf=s_conf)
obj = SummaryStat(sub_df, summary_conf=s_conf)

#obj.describe()
sumstats = obj.generate_summary()
cols_to_cluster = obj.get_new_col_names()
print cols_to_cluster
sumstats.show()
sumstats.count()
#clusters = cluster_df(sumstats, cols_to_cluster, 5)
#clusters.clusterCenters

['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']
+------------------+-----------+-----------+--------------+
|  avg_"heart_rate"|avg_"speed"|"workoutid"|avg_"altitude"|
+------------------+-----------+-----------+--------------+
|             103.0|        0.0|  165504204|          null|
|150.06666666666666|   10.59552|  167479013|          null|
+------------------+-----------+-----------+--------------+



2

In [10]:
r_obj = cluster_df(sumstats, cols_to_cluster, num_clusters=2) 
kmodel = r_obj[0]
new_df = r_obj[1]
scaler_model_dict = r_obj[2]

In [11]:
print denormalize_centers(kmodel.clusterCenters(), scaler_model_dict)
print cols_to_cluster


[[103.0, 0.0, 0.0], [150.06666666666666, 10.59552, 0.0]]
['avg_"heart_rate"', 'avg_"speed"', 'avg_"altitude"']


# ---testing area